# Temporal-Difference Learning

*Temporal-difference* (TD) learning is a combination of Monte Carlo ideas and dynamic programming (DP) ideas.

  * Like Monte Carlo methods, TD methods can learn directly from raw experience without a model of environment's dynamics.
  * Like DP, TD methods update estimates based in part on other learned estimates, without waiting for a final outcome (they bootstrap).

## TD Prediction

### *Constant-$\alpha$ MC*

\begin{align*}
V(S_t) \gets V(S_t) + \alpha[G_t - V(S_t)]
\end{align*}

MC methods must wait until the end of the episode to determine the increment to $V(S_t)$ (only then is $G_t$ is known).

### The simplest TD method

\begin{align*}
V(S_t)      &\gets V(S_t) + \alpha[R_{t+1} + \gamma V(S_{t+1}) - V(S_t)] \\
Q(S_t, A_t) &\gets Q(S_t, A_t) + \alpha [R_{T+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)]
\end{align*}

TD methods need to wait only until the next time step.

### TD error

\begin{align*}
\delta_t \doteq R_{t+1} + \gamma V(S_{t+1}) - V(S_t)
\end{align*}

In [4]:
from collections import defaultdict
import numpy as np
import gym

def td_0_prediction(policy, env, num_episodes, alpha=0.05, gamma=0.99):
    # a dict of (state -> value)
    v = defaultdict(float)
    
    for _ in range(num_episodes):

        state = env.reset()
        done = False
        
        while not done:
            action = policy(state)
            next_state, reward, done, _ = env.step(action)
            v[state] += alpha*(reward + gamma*v[next_state] - v[state])
            state = next_state
            
    return v

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt



In [5]:
env = gym.make('Blackjack-v0')
policy = lambda _state: env.action_space.sample()
v = td_0_prediction(policy, env, 1000)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x1 = []

### $n$-step TD for estimating $V \approx v_\pi$

In [8]:
def n_step_td_prediction(
        policy, env, num_episodes, n_steps=1, alpha=0.05, gamma=0.99):
    # state value: a dict (state -> value)
    v = defaultdict(float)
    
    for _ in range(num_episodes):
        
        state = env.reset()
        done = False
        
        last_steps = []
        
        while not done:
            action = policy(state)
            next_state, reward, done, _ = env.step(action)
            last_steps.append((state, action, reward))
            if done:
                v[next_state] = 0
            if len(last_steps) == n_steps or done:
                v[last_steps[0][0]] = v[last_steps[0][0]] + alpha * (
                    np.sum([gamma**i * r for i, (_, _, r) in enumerate(last_steps)])
                    + v[next_state] - v[last_steps[0][0]])
                last_steps = last_steps[1:]
            state = next_state
            
    return v

In [9]:
env = gym.make('Blackjack-v0')
policy = lambda _state: env.action_space.sample()
v = n_step_td_prediction(policy, env, 10, 2)